In [1]:
from IPython.display import Markdown, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import traceback
import numpy as np
import scipy.stats as ss
import yaml
import itertools
import sys
import os
import subprocess
from os import path
import pickle
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from sklearn.linear_model import LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

def printmd(*args):
    display(Markdown(' '.join(map(str, args))))

In [2]:
# !ros2 run local_planning_performance_modelling compute_metrics -r "~/ds/performance_modelling/output/test_local_planning_gen_circles/*" -s

In [3]:
# !ros2 run local_planning_performance_modelling compute_metrics -r "~/ds/performance_modelling/output/test_local_planning/*" -s

In [4]:
local_planner_color = {'dwa': 'blue', 'arena': 'orange', 'teb': 'green'}
local_planner_symbol = {'dwa': 'o', 'arena': '^', 'teb': 'x'}
robot_model_color = {'hunter': 'cyan', 'turtle': 'green'}
pd.options.display.width = 200
pd.options.display.max_rows = 0
pd.options.display.float_format = '{:,.3f}'.format
plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams['lines.linewidth'] = 2


In [5]:
%run load_data.py

NameError: name 'pd' is not defined

In [6]:
df

NameError: name 'df' is not defined

In [ ]:
# df = pd.read_csv(path.expanduser("~/ds/performance_modelling/output/local_planning/results.csv"))
# with open(path.expanduser("~/ds/performance_modelling/output/local_planning/results_info.yaml")) as results_info_file:
#     results_info = yaml.safe_load(results_info_file)

# # df = pd.read_csv(path.expanduser("~/ds_alt/performance_modelling/output_plot_no_pedestrian_data/results.csv"))
# # with open(path.expanduser("~/ds_alt/performance_modelling/output_plot_no_pedestrian_data/results_info.yaml")) as results_info_file:
# #     results_info = yaml.safe_load(results_info_file)
    
# df.rename(inplace=True, columns={
#     'collisionless_localization_update_absolute_translation_error_mean': 'absolute_translation_error',
#     'collisionless_localization_update_absolute_rotation_error_mean': 'absolute_rotation_error',
#     'collisionless_localization_update_normalized_relative_translation_error_mean': 'normalized_relative_translation_error',
#     'collisionless_localization_update_normalized_relative_rotation_error_mean': 'normalized_relative_rotation_error',
#     'localization_update_rate_mean': 'localization_update_rate',
# })

# # turn odometry_error into beta_1..4
# results_info['run_parameter_names'] += ['beta_1', 'beta_2', 'beta_3', 'beta_4']
# results_info['run_parameter_names'].remove('odometry_error')
# for i in range(0, 4):
#     df[f'beta_{i+1}'] = df['odometry_error'].apply(lambda x: eval(x)[i])
# del df['odometry_error']

# df.loc[df.robot_model == 'turtlebot3_waffle_performance_modelling', 'robot_model'] = 'turtle'

# run_parameters = [c for c in list(df.columns) if c in results_info['run_parameter_names']]
# run_parameters += ['max_steering_angle_deg']
# metrics_versions = [c for c in list(df.columns) if '_version' in c]
# everything_else = ['run_id', 'session_id', 'run_number', 'goal_index']
# metrics = [c for c in df.columns if c not in metrics_versions + run_parameters + everything_else]
# metrics_and_versions = [c for c in list(df.columns) if '_version' in c or c in metrics]

# cpu_time_metrics = [c for c in metrics if 'cpu_time' in c]
# max_memory_metrics = [c for c in metrics if 'max_memory' in c]

# # add useful parameters
# df['session_id'] =  df['run_id'].apply(lambda x:  x.split('_')[1]+'_'+x.split('_')[2]+'_'+x.split('_')[3])
# df['run_number'] =  df['run_id'].apply(lambda x:  int(x.split('_')[5]))
# df[max_memory_metrics] = df[max_memory_metrics]/1024**2
# df['max_steering_angle_deg'] = 90    # crea una nuova colonna e riempie le righe con il valore 90
# df["run_index_str"] = df['run_index'].apply(lambda x: str(x))
# df['goal_index'] = df.environment_name + '_' + df.run_index_str

# # add metrics from existing ones
# df['average_velocity'] = df['trajectory_length'] / df['execution_time']
# df['success_rate'] = df['success_rate'] & (1 - df['collision_rate'])

# metrics += ['average_velocity']
# metrics_and_versions += ['average_velocity']

# min_execution_time_group_df = df.groupby(["environment_name", "run_index", "success_rate"])
# for (environment_name, run_index, success_rate), group_df in min_execution_time_group_df:
#     df.loc[(df.environment_name == environment_name) & (df.run_index == run_index) & (success_rate), 'min_execution_time'] = group_df.execution_time.min()
# df['norm_execution_time'] = df.execution_time / df.min_execution_time
# metrics += ['norm_execution_time']
# metrics_and_versions += ['norm_execution_time']

# if True:
#     printmd("## Number of Runs")
#     printmd(f"    {len(df.run_id.unique())}")

#     printmd("## Run Parameters")
#     for name in [run_parameter_name for run_parameter_name in run_parameters if 'localization_generator_' not in run_parameter_name]:
#         values = list(df[name].unique())
#         printmd(f"    {name:<70}", sorted(values))

#     printmd("## Metrics")
#     for name in metrics_and_versions:
#         if name in metrics_versions:
#             if len(df[name].unique()) == 1:
#                 printmd(f"    {name:<70} {sorted(df[name].unique())}")
#             else:
#                 printmd(f"<code><font style='background-color:yellow;font-family:monospace'>{name:<70}{sorted(df[name].unique())} </font></code> ⚠️")
#         else:
#             printmd(f"    {name:<70} min: {df[name].min(skipna=True):10.4g} {'avg':>15}: {df[name].mean(skipna=True):10.4g} {'max':>15}: {df[name].max(skipna=True):10.4g} {'nan':>15}: {sum(df[name].isna()):10.4g}")

In [ ]:
df.execution_time.agg(['mean', 'count'])

In [ ]:
(df.execution_time.mean() * 1400) / 3600 / 8

In [ ]:
df.groupby(by=['include_costmap_obstacles', 'include_dynamic_obstacles', 'beta_1']).agg(['mean', 'count'])[['success_rate', 'move_base_cpu_time']].sort_values(by=('success_rate', 'mean'), ascending=False)

In [ ]:
import matplotlib.patches as patches

df_success_cpu_trade_off = df.groupby(by=['include_costmap_obstacles', 'include_dynamic_obstacles', 'beta_1']).agg('mean')[['success_rate', 'move_base_cpu_time']].sort_values(by='success_rate', ascending=False)

fig, ax = plt.subplots()

x = list(df_success_cpu_trade_off.move_base_cpu_time)
y = list(df_success_cpu_trade_off.success_rate)
labels = list(df_success_cpu_trade_off.index)

for x_i, y_i in zip(x, y):
    plt.plot([x_i, x_i+100], [y_i, y_i], color='black', linewidth=1, alpha=0.5)
    plt.plot([x_i, x_i], [-100, y_i], color='black', linewidth=1, alpha=0.5)

for i, label in enumerate(labels):
    ax.annotate(label, xy=(x[i], y[i]), xytext=(-24, 6) , xycoords='data', textcoords='offset points')

ax.scatter(df_success_cpu_trade_off.move_base_cpu_time, df_success_cpu_trade_off.success_rate, label=', '.join(df_success_cpu_trade_off.index.names).replace('_', ' '))
plt.xlabel('CPU time [s]')
plt.ylabel('Success rate')
plt.xlim([0, 1.1*df_success_cpu_trade_off.move_base_cpu_time.max()])
plt.ylim([0, 1.1])
plt.title("Trade-off between TEB parameter values")

plt.legend()
plt.grid()
plt.show()



In [ ]:
df.groupby(by=['min_turning_radius', 'include_costmap_obstacles', 'include_dynamic_obstacles']).agg(['mean', 'count'])[['success_rate', 'collision_rate', 'move_base_cpu_time', 'move_base_max_memory']].sort_values(by=('success_rate', 'mean'), ascending=False)

In [ ]:
for local_planner_node in df.local_planner_node.unique():
    df_group = df[df.local_planner_node == local_planner_node].groupby(by='pedestrian_number')
    # raggruppiamo in base alla x e fai la media del collision rate
    df_success_rate = df_group.agg('mean').success_rate
    
    plt.plot(df_success_rate.index, df_success_rate, local_planner_symbol[local_planner_node] + '-', label=local_planner_node, color=local_planner_color[local_planner_node])
    
plt.xticks(sorted(list(set(range(0, 101, 20)).union(set([5])))))
plt.legend()
plt.xlabel('Pedestrian Number')
plt.ylabel('Success Rate')
plt.title('1) Pedestrian Number vs Success Rate')
plt.grid()
plt.show()

In [ ]:
for local_planner_node in df.local_planner_node.unique():
    df_group = df[df.local_planner_node == local_planner_node].groupby(by='pedestrian_number')
    # raggruppiamo in base alla x e fai la media del collision rate
    df_collision_rate = df_group.agg('mean').collision_rate
    
    plt.plot(df_collision_rate.index, df_collision_rate, local_planner_symbol[local_planner_node] + '-', label=local_planner_node, color=local_planner_color[local_planner_node])
    
plt.xticks(sorted(list(set(range(0, 101, 20)).union(set([5])))))
plt.legend()
plt.xlabel('Pedestrian Number')
plt.ylabel('Collision Rate')
plt.title('1) Pedestrian Number vs Collision Rate')
plt.grid()
plt.show()

In [ ]:
for local_planner_node in df.local_planner_node.unique():
    df_group = df[df.local_planner_node == local_planner_node].groupby(by='pedestrian_max_vel')
    # raggruppiamo in base alla x e fai la media del collision rate
    df_collision_rate = df_group.agg('mean').collision_rate
    
    plt.plot(df_collision_rate.index, df_collision_rate, local_planner_symbol[local_planner_node] + '-', label=local_planner_node, color=local_planner_color[local_planner_node])
    
plt.legend()
plt.xlabel('Pedestrian Velocity')
plt.ylabel('Collision Rate')
plt.title('2) Pedestrian Velocity vs Collision Rate')
plt.grid()
plt.show()

In [ ]:
for local_planner_node in df.local_planner_node.unique():
    df_group = df[df.local_planner_node == local_planner_node].groupby(by='pedestrian_number')
    # raggruppiamo in base alla x e fai la media del collision rate
    df_execution_time = df_group.agg('mean').execution_time
    
    plt.plot(df_execution_time.index, df_execution_time, local_planner_symbol[local_planner_node] + '-', label=local_planner_node, color=local_planner_color[local_planner_node])
    
plt.xticks(sorted(list(set(range(0, 101, 20)).union(set([5])))))
plt.legend()
plt.xlabel('Pedestrian Number')
plt.ylabel('Execution Time')
plt.title('3) Pedestrian Number vs Execution Time')
plt.grid()
plt.show()

In [ ]:
for local_planner_node in df.local_planner_node.unique():
    df_group = df[df.local_planner_node == local_planner_node].groupby(by='pedestrian_max_vel')
    # raggruppiamo in base alla x e fai la media del collision rate
    df_execution_time = df_group.agg('mean').execution_time
    
    plt.plot(df_execution_time.index, df_execution_time, local_planner_symbol[local_planner_node] + '-', label=local_planner_node, color=local_planner_color[local_planner_node])
    
plt.legend()
plt.xlabel('Pedestrian Velocity')
plt.ylabel('Execution Time')
plt.title('4) Pedestrian Velocity vs Execution time')
plt.grid()
plt.show()

In [ ]:
for local_planner_node in df.local_planner_node.unique():
    df_group = df[df.local_planner_node == local_planner_node].groupby(by='pedestrian_number')
    # raggruppiamo in base alla x e fai la media del collision rate
    df_execution_time = df_group.agg('mean').trajectory_length
    
    plt.plot(df_execution_time.index, df_execution_time, local_planner_symbol[local_planner_node] + '-', label=local_planner_node, color=local_planner_color[local_planner_node])
    
plt.xticks(sorted(list(set(range(0, 101, 20)).union(set([5])))))    
plt.legend()
plt.xlabel('Pedestrian Velocity')
plt.ylabel('Trajectory Length')
plt.title('5) Pedestrian Velocity vs Trajectory Length')
plt.grid()
plt.show()

In [ ]:
printmd("## Overall performance values by robot model, local planner component, kinematic constraints, and localization component")
printmd("### Considering all runs (sorted by collisionless success rate)")
df.groupby(['robot_model', 'local_planner_node', 'max_steering_angle_deg', 'localization_node'])[["success_rate", "collision_rate", "average_velocity", "norm_execution_time"]].agg(['mean', 'count']).sort_values(('success_rate', 'mean'), ascending=False)

In [ ]:
printmd("### Only considering successful runs (sorted by norm execution time)")
df[df.success_rate == 1].groupby(['robot_model', 'local_planner_node', 'max_steering_angle_deg', 'localization_node'])[["success_rate", "collision_rate", "average_velocity", "norm_execution_time"]].agg(['mean', 'count']).sort_values(('norm_execution_time', 'mean'), ascending=True)

In [ ]:
printmd("### Only considering failed runs (sorted by collision rate)")
df[df.success_rate == 0].groupby(['robot_model', 'local_planner_node', 'max_steering_angle_deg', 'localization_node'])[["success_rate", "collision_rate", "average_velocity", "norm_execution_time"]].agg(['mean', 'count']).sort_values(('collision_rate', 'mean'), ascending=True)

In [ ]:
def bar_plot(metric_name, selected_df, y_columns=['local_planner_node'], percentage=False, unit_symbol=None, higher_better=True):
    bar_df = selected_df.groupby(y_columns)[metric_name].agg(['mean', 'count']).sort_values(by=('mean'), ascending=higher_better)
    ax = (bar_df.sort_values(by=('mean'), ascending=higher_better)* (100 if percentage else 1) ).plot.barh(y=[('mean')], stacked=True, figsize=[10, 5], grid=True, legend=False)
    ax.set_xlabel(metric_name.replace('_', ' ').replace(',', ', ') + (f" [{unit_symbol}]" if unit_symbol else ""))
    ax.set_ylabel(ax.get_ylabel().replace('_', ' ').replace(',', ',\n'))
    plt.gcf().set_size_inches([5, 1])
    plt.show()
# printmd("## success rate")
# bar_plot(metric_name='success_rate', selected_df=df[(df.max_steering_angle_deg == 90.0)], percentage=True, higher_better=True, unit_symbol='%')
# printmd("## collision_rate")
# bar_plot(metric_name='collision_rate', selected_df=df[(df.max_steering_angle_deg == 90.0)], percentage=True, higher_better=False, unit_symbol='%')
# printmd("## average_velocity, only successful runs")
# bar_plot(metric_name='average_velocity', selected_df=df[(df.success_rate == 1) & (df.max_steering_angle_deg == 90.0)], higher_better=True, unit_symbol='m/s')
# printmd("## norm_execution_time, only successful runs")
# bar_plot(metric_name='norm_execution_time', selected_df=df[(df.success_rate == 1) & (df.max_steering_angle_deg == 90.0)], higher_better=False, unit_symbol='s/s')
printmd("## teb only, success rate")
bar_plot(metric_name='success_rate', selected_df=df[(df.local_planner_node == 'teb')], y_columns=['include_costmap_obstacles', 'include_dynamic_obstacles'], percentage=True, higher_better=True, unit_symbol='%')
printmd("## teb only, collision rate")
bar_plot(metric_name='collision_rate', selected_df=df[(df.local_planner_node == 'teb')], y_columns=['include_costmap_obstacles', 'include_dynamic_obstacles'], percentage=True, higher_better=True, unit_symbol='%')
printmd("## teb only, cpu usage")
bar_plot(metric_name='system_max_memory', selected_df=df[(df.local_planner_node == 'teb')], y_columns=['include_costmap_obstacles', 'include_dynamic_obstacles'], higher_better=True, unit_symbol='Mb')
printmd("## teb only, cpu time")
bar_plot(metric_name='move_base_cpu_time', selected_df=df[(df.local_planner_node == 'teb')], y_columns=['include_costmap_obstacles', 'include_dynamic_obstacles'], higher_better=True, unit_symbol='s')

In [ ]:
max_scatter_points = 200
def plot_in_out_metrics_bin_width_save(output_metric, input_metrics, bin_widths, min_bin_count, selected_df, out_dir, min_y=0.0, plot_all=True, scatter=True):
    max_y = -np.inf
#     scatter = False

    for i, (input_metric, bin_width) in enumerate(zip(input_metrics, bin_widths)):
        fig = plt.figure()

        for localization_node in df.localization_node.unique():
            for local_planner_node in df.local_planner_node.unique():
                
                df_lr = selected_df[
                    (selected_df.localization_node == localization_node) & 
                    (selected_df.local_planner_node == local_planner_node) & 
                    selected_df[input_metric.name].notna() & 
                    selected_df[output_metric.name].notna()
                ].copy()
                if len(df_lr[input_metric.name]):
                    if scatter:
                        n = max(1, int(len(df_lr[input_metric.name])/max_scatter_points))
                        plt.scatter(df_lr[input_metric.name][::n], df_lr[output_metric.name][::n], marker='.', s=0.25, color=local_planner_color[local_planner_node])
                    
                    if bin_width is not None:
                        if isinstance(bin_width, str):
                            _, bins = np.histogram(df_lr[input_metric.name], bins=bin_width)
                        else:
                            bins = np.arange(0., input_metric.max() + bin_width, bin_width)

                        df_lr['x_range'] = pd.cut(df_lr[input_metric.name], bins, labels=bins[:-1]).astype('float')
                        df_lr_g = df_lr.groupby('x_range').agg(['mean', 'count'])
                        df_lr_g = df_lr_g[ df_lr_g[(output_metric.name, 'count')] > min_bin_count ]
                        plt.plot(df_lr_g.index, df_lr_g[(output_metric.name, 'mean')], f'{local_planner_symbol[local_planner_node]}-', fillstyle='none', mfc=None, label=f'{local_planner_node}', linewidth=1, color=local_planner_color[local_planner_node])
                    else:
                        df_lr_g = df_lr.groupby(input_metric.name).agg(['mean', 'count'])
                        df_lr_g = df_lr_g[ df_lr_g[(output_metric.name, 'count')] > min_bin_count ]
                        plt.plot(df_lr_g.index, df_lr_g[(output_metric.name, 'mean')], f'{local_planner_symbol[local_planner_node]}-', fillstyle='none', mfc=None, label=f'{local_planner_node}', linewidth=1, color=local_planner_color[local_planner_node])
                    max_y = max(max_y, df_lr_g[(output_metric.name, 'mean')].max())

        if i == 0:
            plt.legend()
        plt.grid()
        plt.xlabel(input_metric.name.replace('_', ' '))
        plt.ylabel(output_metric.name.replace('_', ' '))
        plt.ylim([min_y, max_y*1.1])
        fig.set_size_inches([4, 4])
        if not path.exists(out_dir):
            os.makedirs(out_dir)
        fig.savefig(out_dir + f"/{output_metric.name}---{input_metric.name}.pdf".replace('_', '-'), bbox_inches='tight')
        plt.show()


In [ ]:
df.groupby(by=['pedestrian_number', 'local_planner_node']).agg(['count', 'mean'])[['simulation_max_memory', 'simulation_cpu_time', 'system_cpu_time', 'system_max_memory', 'move_base_cpu_time', 'move_base_max_memory']].sort_values(by=('pedestrian_number'), ascending=False)

In [ ]:
df[].groupby(by=['pedestrian_number', 'local_planner_node']).agg(['count', 'mean'])[['real_time_factor']].sort_values(by=('pedestrian_number'), ascending=False)

In [ ]:
df[df.local_planner_node == 'arena'][df.pedestrian_number == 100]['real_time_factor']

In [ ]:
df[df.local_planner_node == 'teb'][df.pedestrian_number == 100][df.real_timefa]['real_time_factor']

In [ ]:
df[df.real_time_factor > 1]

In [ ]:
df[df.real_time_factor > 1].real_time_factor